## A. Build a baseline model (5 marks)

Use the Keras library to build a neural network with the following:

One hidden layer of 10 nodes, and a ReLU activation function

Use the adam optimizer and the mean squared error as the loss function.

Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.

Train the model on the training data using 50 epochs.

Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

Report the mean and the standard deviation of the mean squared errors.

Submit your Jupyter Notebook with your code and comments.

In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [10]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>


## Import Keras


Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.


#### Let's go ahead and import the Keras library


In [11]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [12]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Part A: Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [13]:
# define regression model
def regression_modelA():
    # create model
    modelA = Sequential()
    modelA.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    modelA.add(Dense(1))
    
    # compile model
    modelA.compile(optimizer='adam', loss='mean_squared_error')
    return modelA

The above function create a model that has two hidden layers, each of 50 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [14]:
# build the model
modelA = regression_modelA()

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.


In [24]:
from sklearn.model_selection import train_test_split
A_mean_squared_errors = []
for epoch in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3)
    res = modelA.fit(X_train, y_train, epochs=50, verbose=0, validation_data=(X_test, y_test))
    mean_squared_error = res.history['val_loss'][-1]
    A_mean_squared_errors.append(mean_squared_error)
    print('Epoch #{}: mean_squared_error {}'.format(epoch+1, mean_squared_error))

2023-07-29 01:21:42.206165: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2023-07-29 01:21:42.211954: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394310000 Hz
2023-07-29 01:21:42.212625: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556bb7acf9f0 executing computations on platform Host. Devices:
2023-07-29 01:21:42.212678: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2023-07-29 01:21:42.352267: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

Epoch #1: mean_squared_error 106.67502319928512
Epoch #2: mean_squared_error 74.06504462529155
Epoch #3: mean_squared_error 68.45697460977004
Epoch #4: mean_squared_error 78.45342405251314
Epoch #5: mean_squared_error 59.90618280453975
Epoch #6: mean_squared_error 74.07556199255885
Epoch #7: mean_squared_error 55.091871057899255
Epoch #8: mean_squared_error 56.253151779421714
Epoch #9: mean_squared_error 55.467234972611216
Epoch #10: mean_squared_error 48.437109962636214
Epoch #11: mean_squared_error 50.20719480051578
Epoch #12: mean_squared_error 54.04427314190417
Epoch #13: mean_squared_error 48.977801400095125
Epoch #14: mean_squared_error 52.02880165569219
Epoch #15: mean_squared_error 45.6756617968908
Epoch #16: mean_squared_error 41.871681509666075
Epoch #17: mean_squared_error 47.770798272685326
Epoch #18: mean_squared_error 54.69599404689949
Epoch #19: mean_squared_error 52.01846677662871
Epoch #20: mean_squared_error 42.887261153039034
Epoch #21: mean_squared_error 45.67875573

In [27]:
mean_value = np.mean(A_mean_squared_errors)
std_value = np.std(A_mean_squared_errors)

print(f'The mean of the mean squared errors: {mean_value}')
print(f'The standard deviation of the mean squared errors: {std_value}')

The mean of the mean squared errors: 52.75061288913863
The standard deviation of the mean squared errors: 11.196211206377388


## Part B: Normalize the Data

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?

Normalize the data by substracting the mean and dividing by the standard deviation:

In [28]:
predictors_norm

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069
...,...,...,...,...,...,...,...,...
1025,-0.045623,0.487998,0.564271,-0.092126,0.451190,-1.322363,-0.065861,-0.279597
1026,0.392628,-0.856472,0.959602,0.675872,0.702285,-1.993711,0.496651,-0.279597
1027,-1.269472,0.759210,0.850222,0.521336,-0.017520,-1.035561,0.080068,-0.279597
1028,-1.168042,1.307430,-0.846733,-0.279443,0.852942,0.214537,0.191074,-0.279597


In [29]:
n_cols = predictors_norm.shape[1]
def regression_modelB():
    modelB = Sequential()
    modelB.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    modelB.add(Dense(1))
    
    modelB.compile(optimizer='adam', loss='mean_squared_error')
    return modelB

In [30]:
modelB = regression_modelB()

In [31]:
from sklearn.model_selection import train_test_split
B_mean_squared_errors = []
for epoch in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    res = modelB.fit(X_train, y_train, epochs=50, verbose=0, validation_data=(X_test, y_test))
    mean_squared_error = res.history['val_loss'][-1]
    B_mean_squared_errors.append(mean_squared_error)
    print('Epoch #{}: mean_squared_error {}'.format(epoch+1, mean_squared_error))

Epoch #1: mean_squared_error 485.56095140574433
Epoch #2: mean_squared_error 153.24887460258014
Epoch #3: mean_squared_error 122.52653686128387
Epoch #4: mean_squared_error 106.1128368686318
Epoch #5: mean_squared_error 74.15961778511121
Epoch #6: mean_squared_error 65.33269859980611
Epoch #7: mean_squared_error 54.851417467432114
Epoch #8: mean_squared_error 41.974890860153245
Epoch #9: mean_squared_error 51.271254382087186
Epoch #10: mean_squared_error 41.52644313577695
Epoch #11: mean_squared_error 43.077328073939846
Epoch #12: mean_squared_error 36.286948429342225
Epoch #13: mean_squared_error 34.11360663503505
Epoch #14: mean_squared_error 30.581960486748457
Epoch #15: mean_squared_error 31.564379392704147
Epoch #16: mean_squared_error 36.54071673297573
Epoch #17: mean_squared_error 37.1490540242118
Epoch #18: mean_squared_error 32.60346435497494
Epoch #19: mean_squared_error 31.30384952582202
Epoch #20: mean_squared_error 37.271214247521456
Epoch #21: mean_squared_error 37.780114

In [32]:
mean_value = np.mean(B_mean_squared_errors)
std_value = np.std(B_mean_squared_errors)

print(f'The mean of the mean squared errors: {mean_value}')
print(f'The standard deviation of the mean squared errors: {std_value}')

The mean of the mean squared errors: 49.87950074729796
The standard deviation of the mean squared errors: 66.72749621161523


The mean and standard deviation in comparison to A is both fairly less, indicating that normalizing the data helped reduced the error. 

# Part C: Increase the number of epochs

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?

Train and test the model at the same time using the fit-method. We will leave out 30% of the data (data after normalization) for validation and we will train the model for 100 epochs instead of 50 epochs.

In [33]:
def regression_modelC():
    modelC = Sequential()
    modelC.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    modelC.add(Dense(1))
    
    modelC.compile(optimizer='adam', loss='mean_squared_error')
    return modelC

In [34]:
modelC = regression_modelC() 

In [35]:
from sklearn.model_selection import train_test_split
C_mean_squared_errors = []
for epoch in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    res = modelC.fit(X_train, y_train, epochs=50, verbose=0, validation_data=(X_test, y_test))
    mean_squared_error = res.history['val_loss'][-1]
    C_mean_squared_errors.append(mean_squared_error)
    print('Epoch #{}: mean_squared_error {}'.format(epoch+1, mean_squared_error))

Epoch #1: mean_squared_error 315.45540236192227
Epoch #2: mean_squared_error 147.41360369932303
Epoch #3: mean_squared_error 140.32763010166997
Epoch #4: mean_squared_error 104.28987469719452
Epoch #5: mean_squared_error 97.76742452479489
Epoch #6: mean_squared_error 92.54551262222833
Epoch #7: mean_squared_error 80.94648362440584
Epoch #8: mean_squared_error 80.83635546242921
Epoch #9: mean_squared_error 70.3571030576638
Epoch #10: mean_squared_error 68.60681754794321
Epoch #11: mean_squared_error 57.690255149668474
Epoch #12: mean_squared_error 52.52209431916765
Epoch #13: mean_squared_error 41.578856570049396
Epoch #14: mean_squared_error 55.12346061533709
Epoch #15: mean_squared_error 45.56177949133814
Epoch #16: mean_squared_error 56.444063896500175
Epoch #17: mean_squared_error 47.40389754162637
Epoch #18: mean_squared_error 41.65774442308543
Epoch #19: mean_squared_error 45.96033459265255
Epoch #20: mean_squared_error 49.12250486701052
Epoch #21: mean_squared_error 46.2865227511

In [36]:
mean_value = np.mean(C_mean_squared_errors)
std_value = np.std(C_mean_squared_errors)

print(f'The mean of the mean squared errors: {mean_value}')
print(f'The standard deviation of the mean squared errors: {std_value}')

The mean of the mean squared errors: 60.112588462953056
The standard deviation of the mean squared errors: 43.4823514534679


It seems like the mean and standard deviation are both very similar in comparison to part B.

# Part D: Increase the number of hidden layers

Repeat part B but use a neural network with the following instead:

Three hidden layers, each of 10 nodes and ReLU activation function.
How does the mean of the mean squared errors compare to that from Step B?

Create a new model with three hidden layers, each of 10 nodes and ReLU activation function.

In [37]:
def regression_modelD():
    modelD = Sequential()
    modelD.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    modelD.add(Dense(10, activation='relu'))
    modelD.add(Dense(10, activation='relu'))
    modelD.add(Dense(1))
    
    modelD.compile(optimizer='adam', loss='mean_squared_error')
    return modelD

In [38]:
modelD = regression_modelD()

In [39]:
from sklearn.model_selection import train_test_split
D_mean_squared_errors = []
for epoch in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3)
    res = modelD.fit(X_train, y_train, epochs=50, verbose=0, validation_data=(X_test, y_test))
    mean_squared_error = res.history['val_loss'][-1]
    D_mean_squared_errors.append(mean_squared_error)
    print('Epoch #{}: mean_squared_error {}'.format(epoch+1, mean_squared_error))

Epoch #1: mean_squared_error 114.17832292476518
Epoch #2: mean_squared_error 69.4446599892428
Epoch #3: mean_squared_error 44.02836284205366
Epoch #4: mean_squared_error 40.69353802381596
Epoch #5: mean_squared_error 30.426959386535447
Epoch #6: mean_squared_error 27.150615827936956
Epoch #7: mean_squared_error 25.934255686392675
Epoch #8: mean_squared_error 31.207246564352783
Epoch #9: mean_squared_error 28.153956638570744
Epoch #10: mean_squared_error 28.219566758782346
Epoch #11: mean_squared_error 27.55519253999284
Epoch #12: mean_squared_error 26.90124395981576
Epoch #13: mean_squared_error 27.3513232049047
Epoch #14: mean_squared_error 25.33335390060079
Epoch #15: mean_squared_error 25.718531932645632
Epoch #16: mean_squared_error 27.92875529415785
Epoch #17: mean_squared_error 22.22807047205064
Epoch #18: mean_squared_error 23.364294184835984
Epoch #19: mean_squared_error 26.817403250141822
Epoch #20: mean_squared_error 23.72879419666278
Epoch #21: mean_squared_error 24.39992586

In [40]:
mean_value = np.mean(D_mean_squared_errors)
std_value = np.std(D_mean_squared_errors)

print(f'The mean of the mean squared errors: {mean_value}')
print(f'The standard deviation of the mean squared errors: {std_value}')

The mean of the mean squared errors: 27.329474804578858
The standard deviation of the mean squared errors: 14.67119244830501


It seems like this was the lowest error produced in comparison between part A, B, and C. 